In [71]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import csv as csv
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [72]:
pwd

'C:\\Users\\Jared Chung\\Desktop\\kaggle Comp\\Titanic'

In [73]:
ls

 Volume in drive C is Windows
 Volume Serial Number is CAA3-57F9

 Directory of C:\Users\Jared Chung\Desktop\kaggle Comp\Titanic

04/10/2015  11:50 AM    <DIR>          .
04/10/2015  11:50 AM    <DIR>          ..
27/09/2015  09:41 PM    <DIR>          .ipynb_checkpoints
27/09/2015  09:40 PM             4,176 First_Forest.py
04/10/2015  11:44 AM             3,258 Logistic_regression.csv
04/10/2015  11:29 AM                 0 logistic_res.csv
27/09/2015  09:38 PM            28,629 test.csv
04/10/2015  11:50 AM            19,523 Titanic_Ipython.ipynb
27/09/2015  09:38 PM            61,194 train.csv
               6 File(s)        116,780 bytes
               3 Dir(s)  150,936,895,488 bytes free


In [74]:
train_df = pd.read_csv('train.csv', header=0) 

In [75]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [76]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB


In [77]:
train_df['Gender'] = train_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [78]:
train_df.loc[train_df.Age.isnull(),'Age'] = np.average(train_df.loc[train_df.Age.notnull()].Age)

In [79]:
# All missing Embarked -> just make them embark from most common place
if len(train_df.Embarked[ train_df.Embarked.isnull() ]) > 0:
    train_df.loc[ train_df.Embarked.isnull(),'Embarked'] = train_df.Embarked.dropna().mode().values

Ports = list(enumerate((train_df['Embarked']).unique()))    # determine all values of Embarked,
Ports_dict = { name : i for i, name in Ports }              # set up a dictionary in the form  Ports : index
train_df.Embarked = train_df.Embarked.map( lambda x: Ports_dict[x]).astype(int)     # Convert all Embark strings to int


In [80]:
# Remove the Name column, Cabin, Ticket, and Sex (since I copied and filled it to Gender)
train_df = train_df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'PassengerId'], axis=1) 


In [81]:
# TEST DATA
test_df = pd.read_csv('test.csv', header=0)        # Load the test file into a dataframe



In [82]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [83]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 39.2+ KB


In [84]:
# I need to do the same with the test data now, so that the columns are the same as the training data
# I need to convert all strings to integer classifiers:
# female = 0, Male = 1
test_df['Gender'] = test_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [85]:
# Embarked from 'C', 'Q', 'S'
# All missing Embarked -> just make them embark from most common place
if len(test_df.Embarked[ test_df.Embarked.isnull() ]) > 0:
    test_df.Embarked[ test_df.Embarked.isnull() ] = test_df.Embarked.dropna().mode().values
# Again convert all Embarked strings to int
test_df.Embarked = test_df.Embarked.map( lambda x: Ports_dict[x]).astype(int)

In [86]:
#Fill in missing Age with Average
test_df.loc[test_df.Age.isnull(),'Age'] = np.average(test_df.loc[test_df.Age.notnull(),'Age'])

In [87]:
#fill in missing Fare... Might need to change this
test_df.loc[test_df.Fare.isnull(),'Fare'] = np.average(test_df.loc[test_df.Fare.notnull(),'Fare'])


In [88]:
# Collect the test data's PassengerIds before dropping it
ids = test_df['PassengerId'].values
# Remove the Name column, Cabin, Ticket, and Sex (since I copied and filled it to Gender)
test_df = test_df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'PassengerId'], axis=1) 

In [89]:
# The data is now ready to go. So lets fit to the train, then predict to the test!
# Convert back to a numpy array
train_data = train_df.values
test_data = test_df.values

In [90]:
print ('Training...')
clf = svm.SVC()
clf = clf.fit( train_data[0::,1::], train_data[0::,0] )

Training...


In [91]:
print ('Predicting...')
output = clf.predict(test_data).astype(int)

Predicting...


In [92]:
d = DataFrame(data=OrderedDict([('PassengerId', ids), ('Survived', output)])) #### Create Dataframe of data
d.to_csv('SVM.csv', index=False)